In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
BASE_PATH = Path('/home/rico/Documents/courses/6.C571/Final Project/ernp/data/Boston')
city = "Boston"
country = "US"
osm_poi_tags = {'amenity':'school'}
poi_file = BASE_PATH / "boston_pois.geojson"
gtfs_file = BASE_PATH / "mbta_transit_feed.zip"
crs = "EPSG:4326"

# Census Data

In [ ]:
# towns_gdf = gpd.read_file(BASE_PATH / "CENSUS2020TOWNS_SHP/CENSUS2020TOWNS_POLY.shp")
# towns_gdf['GEOID20'] = towns_gdf['GEOID20'].astype(str)
# towns_gdf['PLACE_GEOID20'] = towns_gdf['GEOID20'].str[:7].astype(str)
census_blocks_gdf = gpd.read_file(BASE_PATH / 'CENSUS2020_BLK_BG_TRCT/CENSUS2020BLOCKS_POLY.shp')
census_blocks_gdf['GEOID20'] = census_blocks_gdf['GEOID20'].astype(str)
census_bgs_gdf = gpd.read_file(BASE_PATH / 'CENSUS2020_BLK_BG_TRCT/CENSUS2020BLOCKGROUPS_POLY.shp')
census_bgs_gdf['GEOID20'] = census_bgs_gdf['GEOID20'].astype(str)
# census_bgs_gdf['PLACE_GEOID20'] = census_bgs_gdf['GEOID20'].str[:7].astype(str)
merged_df = gpd.sjoin(census_bgs_gdf, census_blocks_gdf[['geometry', 'TOWN']], how='left', predicate='intersects')
merged_df = merged_df[[col for col in merged_df.columns if 'block' not in col]]
merged_df = merged_df[['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLKGRPCE20', 'GEOID20',
                       'NAMELSAD20', 'MTFCC20', 'ALAND20', 'AWATER20', 'INTPTLAT20',
                       'INTPTLON20', 'AREA_SQFT', 'AREA_ACRES', 'TRACT20', 'HOUSING20',
                       'POP20', 'BLK20_CNT', 'SHAPE_AREA', 'SHAPE_LEN', 'TOWN', 'geometry']]
merged_df = merged_df.drop_duplicates(subset='GEOID20')
# merged_df.explore()
# Load the MBTA communities excel file
mbta_communities = pd.read_excel(BASE_PATH / "mbta_communities.xlsx")
mbta_communities['Municipality'] = mbta_communities['Municipality'].str.upper()
filtered_merged_df = merged_df[merged_df['TOWN'].isin(mbta_communities['Municipality'].to_list() + ['BOSTON'])]
filtered_merged_df = filtered_merged_df.merge(mbta_communities[['Municipality', 'MBTA Community Type']], left_on='TOWN',
                                              right_on='Municipality', how='left')
filtered_merged_df = filtered_merged_df.drop(columns='Municipality')
filtered_merged_df.loc[filtered_merged_df['MBTA Community Type'].isna(), 'MBTA Community Type'] = 'subway or light rail'
filtered_merged_df.shape
filtered_merged_df.explore()
filtered_merged_df.head(5)
# ACS data
acs_gdf = pd.read_csv(BASE_PATH / 'acs_2022.csv', skiprows=0)
# Remove first row
acs_gdf = acs_gdf.drop(0)
# Assuming your ACS data is in a DataFrame called `acs_df`
acs_gdf['State FIPS'] = acs_gdf['State (FIPS Code)'].astype(str).str.zfill(2)
acs_gdf['County FIPS'] = acs_gdf['County of current residence'].astype(str).str.zfill(3)
acs_gdf['Census Tract'] = acs_gdf['Census Tract'].astype(str).str.zfill(6)
acs_gdf['Block Group'] = acs_gdf['Block Group'].astype(str).str.zfill(1)

# Create GEOID20 for Block Group
acs_gdf['GEOID20'] = (
        acs_gdf['State FIPS'] +
        acs_gdf['County FIPS'] +
        acs_gdf['Census Tract'] +
        acs_gdf['Block Group']
)
str_columns = [
    'Qualifying Name',
    'GEOID20',
]

int_columns = [
    # Denomenator for income categories
    'Households:',
    'Households: Less than $25,000',
    'Households: $25,000 to $49,999',
    'Households: $50,000 to $74,999',
    'Households: $75,000 to $99,999',
    'Households: $100,000 or More',

    # This is the total population of the census tract (count)
    'Total Population',
    # Denominator for age categories beginning with male
    'Total Population: Male',
    'Total Population: Male: Under 18 Years',
    'Total Population: Male: 18 to 34 Years',
    'Total Population: Male: 35 to 64 Years',
    'Total Population: Male: 65 Years and Over',
    # Denominator for age categories beginning with female
    'Total Population: Female',
    'Total Population: Female: Under 18 Years',
    'Total Population: Female: 18 to 34 Years',
    'Total Population: Female: 35 to 64 Years',
    'Total Population: Female: 65 Years and Over',

    # Merging all racial categories within "Hispanic and Latino" so that
    # there aren't an overwhelming number of racial categories 
    'Total Population: Hispanic or Latino',
    'Total Population: Not Hispanic or Latino',
    'Total Population: Not Hispanic or Latino: White Alone',
    'Total Population: Not Hispanic or Latino: Black or African American Alone',
    'Total Population: Not Hispanic or Latino: American Indian and Alaska Native Alone',
    'Total Population: Not Hispanic or Latino: Asian Alone',
    'Total Population: Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone',
    'Total Population: Not Hispanic or Latino: Some Other Race Alone',
    'Total Population: Not Hispanic or Latino: Two or More Races',

    'Workers 16 Years and Over:',
    'Workers 16 Years and Over: Car, Truck, or Van',
    'Workers 16 Years and Over: Drove Alone',
    'Workers 16 Years and Over: Public Transportation (Includes Taxicab)',
    'Workers 16 Years and Over: Motorcycle',
    'Workers 16 Years and Over: Bicycle',
    'Workers 16 Years and Over: Walked',
    'Workers 16 Years and Over: Other Means',
    'Workers 16 Years and Over: Worked At Home',

    'Occupied Housing Units: No Vehicle Available',
    'Occupied Housing Units: 1 Vehicle Available',
    'Occupied Housing Units: 2 Vehicles Available',
]

float_columns = [
    'Area Total:',
    'Area (Land)',
    'Area (Water)',
    'Population Density (Per Sq. Mile)',
    'Median Household Income (In 2022 Inflation Adjusted Dollars)',
]

columns_to_include = str_columns + int_columns + float_columns
filtered_acs_gdf = acs_gdf[columns_to_include]

filtered_acs_gdf.loc[
    filtered_acs_gdf['Population Density (Per Sq. Mile)'].isna(), 'Population Density (Per Sq. Mile)'] = \
filtered_acs_gdf['Total Population'].astype(int) / filtered_acs_gdf['Area Total:'].astype(float)
# filtered_acs_gdf.loc[filtered_acs_gdf['Median Household Income (In 2022 Inflation Adjusted Dollars)'].isna(), 'Median Household Income (In 2022 Inflation Adjusted Dollars)'] = -1
complete_census = filtered_merged_df.merge(filtered_acs_gdf, on='GEOID20', how='left')
# Load your GeoDataFrame
gdf = complete_census  # Assuming this is your loaded GeoDataFrame

# Ensure the column for Median Household Income is numeric
gdf['Median Household Income (In 2022 Inflation Adjusted Dollars)'] = pd.to_numeric(
    gdf['Median Household Income (In 2022 Inflation Adjusted Dollars)'], errors='coerce'
)

# Mark -1 as NaN to identify missing data
gdf.loc[gdf['Median Household Income (In 2022 Inflation Adjusted Dollars)'] == -1,
'Median Household Income (In 2022 Inflation Adjusted Dollars)'] = np.nan

# Find block groups with NaN in the target column
missing_income = gdf[gdf['Median Household Income (In 2022 Inflation Adjusted Dollars)'].isna()]

# Spatial join to find neighbors
gdf['geometry'] = gdf['geometry'].buffer(0)  # Fix potential invalid geometries
# Perform spatial join and ensure correct columns
neighbors = gpd.sjoin(gdf, gdf, how='inner', predicate='touches', lsuffix='left', rsuffix='right')


# Define a function to calculate the average of neighboring block groups
def spatially_fill_missing_values(row, column_name, neighbors_df):
    neighbors_of_row = neighbors_df[neighbors_df.index == row.name]
    neighboring_values = gdf.loc[neighbors_of_row['index_right'], column_name]
    valid_values = neighboring_values[neighboring_values.notna()]
    return valid_values.mean() if not valid_values.empty else 0


# Fill missing income values
gdf['Median Household Income (In 2022 Inflation Adjusted Dollars)'] = gdf.apply(
    lambda row: spatially_fill_missing_values(
        row,
        'Median Household Income (In 2022 Inflation Adjusted Dollars)',
        neighbors
    ) if pd.isna(row['Median Household Income (In 2022 Inflation Adjusted Dollars)'])
    else row['Median Household Income (In 2022 Inflation Adjusted Dollars)'],
    axis=1
)

In [ ]:
# Coerce into types
gdf[str_columns] = gdf[str_columns].astype(str)
gdf[int_columns] = gdf[int_columns].astype(int)
gdf[float_columns] = gdf[float_columns].astype(float)
gdf['res_centroid'] = gpd.points_from_xy(gdf['INTPTLON20'], gdf['INTPTLAT20'], crs=crs)

In [ ]:
gdf.to_parquet(BASE_PATH / 'complete_census_2022.parquet')